In [1]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext


sc = SparkContext(appName='ModelEvaluation').getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession(sparkContext = sc)
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [6]:
from pyspark.ml.regression import RandomForestRegressionModel
randomForest = RandomForestRegressionModel.load('/home/dyllanjr/Solar_Irradiance_Prediction/models/Random_Forest')


In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [11]:
###Testing generalizability on similar region
df = spark.read.csv(f"hdfs://localhost:9000/solar_data/processed/Georgia_1.csv", inferSchema=True, header=True)

cols = df.columns
cols.remove('sum(GHI)')

#vectorize features
assembler = VectorAssembler(inputCols=cols, outputCol='features')
data = assembler.transform(df)


evaluator = RegressionEvaluator(labelCol='sum(GHI)')
test_preds = randomForest.transform(data)
print("Test RMSE: ", evaluator.evaluate(test_preds))
evaluator = RegressionEvaluator(labelCol='sum(GHI)', metricName='r2')
print("Test r2: ", evaluator.evaluate(test_preds))

Test RMSE:  1527.807362271392
Test r2:  0.8587827806769075


In [13]:
###Testing generalizability on different region
df = spark.read.csv(f"hdfs://localhost:9000/solar_data/processed/Arizona_1.csv", inferSchema=True, header=True)

cols = df.columns
cols.remove('sum(GHI)')

#vectorize features
assembler = VectorAssembler(inputCols=cols, outputCol='features')
data = assembler.transform(df)


evaluator = RegressionEvaluator(labelCol='sum(GHI)')
test_preds = randomForest.transform(data)
print("Test RMSE: ", evaluator.evaluate(test_preds))
evaluator = RegressionEvaluator(labelCol='sum(GHI)', metricName='r2')
print("Test r2: ", evaluator.evaluate(test_preds))

Test RMSE:  3344.19744749896
Test r2:  0.34988601334393477
